# Question1 

# Feature Extraction with Classification Model

## Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from matplotlib import*
import matplotlib.pyplot as plt
from scipy import stats
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import time
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
import seaborn
import time
import io

## Extraction of Dataset from Kaggle 

In [ ]:


from google.colab import files
import io
uploaded = files.upload()
!pip install -q kaggle

# Let's make sure the kaggle.json file is present.
!ls -lha kaggle.json

# Next, install the Kaggle API client.
!pip install -q kaggle

# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c ece-657a-w20-asg3-part-1

!unzip train.csv.zip
!unzip testX.csv.zip

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('testX.csv')

### Standardizing and Splitting Dataset and Labels 

In [ ]:
X= train_data.iloc[:, 2:786].values
Y= train_data.iloc[:,1].values
print(X.shape)

X_test= test_data.iloc[:,1:].values
print(X_test.shape)

X_std = StandardScaler().fit_transform(X)
X_std_test = StandardScaler().fit_transform(X_test)
print(X_std.shape)

### Finding explained variance for PCA

In [ ]:
#Explained variance for PCA
pca = PCA(n_components=200, random_state= 42)
X_std = pca.fit_transform(X)
X_test=pca.transform(X_std_test)

pca.n_components=200
explained_variance = pca.explained_variance_ratio_
cum_var_explained=np.cumsum(explained_variance)
print(cum_var_explained)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(cum_var_explained)
plt.axis('tight')  
plt.grid() 
plt.xlabel('n_components') 
plt.ylabel('Cumulative_Variance_explained')  
plt.show()

### Analysis of Hyperparameters for Sample dataset for PCA+ SVM

In [ ]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train1, X_rem1, Y_train1, Y_rem1 = train_test_split(X, Y, test_size=0.83333, random_state=42)
# Standardising the data
X_train1_std = StandardScaler().fit_transform(X_train1)
X_std_test = StandardScaler().fit_transform(X_std_test)
X_train1_std.shape

In [ ]:
param_grid = {'C': [0.1,0.5,1,2,5,10,20,50], 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf']
              
#Implementing PCA with SVM for 10000 samples for 200 features
start_time = time.clock()
pca = PCA(n_components=200, svd_solver='randomized',
whiten=True,random_state=42)
X_train1_pca = pca.fit_transform(X_train1_std)
X_test_pca = pca.fit_transform(X_std_test)
grid = GridSearchCV(SVC(),param_grid,refit=True,verbose=2)
grid.fit(X_train1_pca,Y_train1)
print(grid.best_estimator_)
grid_predictions = grid.predict(X_test_pca)
print(grid_predictions)

time= time.clock() - start_time
print(time)

### Analysis of Hyperparameters for Sample dataset for PCA+RF

In [ ]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train1, X_rem1, Y_train1, Y_rem1 = train_test_split(X, Y, test_size=0.83333, random_stat
(10000, 784)
# Standardising the data
X_train1_std = StandardScaler().fit_transform(X_train1)
X_std_test = StandardScaler().fit_transform(X_std_test)
X_train1_std.shape

In [ ]:
start_time = time.clock()

pca = PCA(n_components=200, svd_solver='randomized',whiten=True,random_state=42)
X_train1_pca = pca.fit_transform(X_train1_std)
X_test_pca = pca.fit_transform(X_std_test)


rf = RandomForestClassifier()
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 1000, num = 100)]
max_features = ['log2', 'sqrt']
max_depth = [int(x) for x in np.linspace(start = 10, stop = 100, num = 10)]
bootstrap = [True, False]


param_dist = {'n_estimators': n_estimators,'max_features': max_features,'max_depth': max_depth,'bootstrap': bootstrap}
print(param_dist)
rs = RandomizedSearchCV(estimator= rf,param_distributions = param_dist,n_iter = 100,
                        cv = 3,verbose = 1,n_jobs=-1,random_state=42)
rs.fit(X_train1_pca, Y_train1)
print(rs.best_params_)

rs_predictions = rs.predict(X_test_pca)
print(rs_predictions)

time= time.clock() - start_time
print(" Time in Seconds: ",time)

### Finding Explained Variance for LDA

In [ ]:
X_std = StandardScaler().fit_transform(X)
X_std_test = StandardScaler().fit_transform(X_test)

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis(n_components=4)
X_lda = lda.fit_transform(X_std,Y)
X_std_test_full = lda.transform(X_std_test)

In [ ]:
lda.n_components=4
explained_variance = lda.explained_variance_ratio_
cum_var_explained=np.cumsum(explained_variance)
print(cum_var_explained)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(cum_var_explained)
plt.axis('tight')
plt.grid()
plt.xlabel('n_components')
plt.ylabel('Cumulative_Variance_explained')
plt.show()

### Analysis of Hyperparameters for Sample dataset for LDA with SVM

In [ ]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train1, X_rem1, Y_train1, Y_rem1 = train_test_split(X, Y, test_size=0.83333, random_stat
(10000, 784)
# Standardising the data
X_train1_std = StandardScaler().fit_transform(X_train1)
X_std_test = StandardScaler().fit_transform(X_test)
X_train1_std.shape

In [ ]:
param_grid = {'C': [0.1,0.5,1,2,5,10,20,50], 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf']
              
#Implementing PCA with SVM for 10000 samples for 200 features
lda = LinearDiscriminantAnalysis(n_components=4)
X_train1_lda = lda.fit_transform(X_train1_std,Y_train1)
X_test_lda = lda.transform(X_std_test)
              
grid = GridSearchCV(SVC(),param_grid,refit=True,verbose=2)
grid.fit(X_train1_lda,Y_train1)
print(grid.best_estimator_)
grid_predictions = grid.predict(X_test_lda)

print(grid_predictions)


### 1. PCA with SVM

#### Default SVM with PCA=200 components

In [ ]:
X= train_data.iloc[:, 2:786].values
Y= train_data.iloc[:,1].values
print(X.shape)

X_test= test_data.iloc[:,1:].values
print(X_test.shape)

X_std = StandardScaler().fit_transform(X)
X_std_test = StandardScaler().fit_transform(X_test)
print(X_std.shape)

In [ ]:
start_time = time.clock()

pca = PCA(n_components=200, svd_solver='randomized',
whiten=True,random_state=42)
pca.fit(X_std)
X_train_pca = pca.transform(X_std)
X_test_pca = pca.transform(X_std_test)
clf = SVC()

scores=cross_val_score(clf, X_train_pca,Y, cv=10)
print(scores)
clf.fit(X_train_pca,Y)
clf_predictions = clf.predict(X_test_pca)
print(clf_predictions)

time= time.clock() - start_time
print(time)

#### PCA=100, SVM(C=60,kernel=rbf)

In [ ]:
start_time = time.clock()

pca = PCA(n_components=100, svd_solver='randomized',
whiten=True,random_state=42)
pca.fit(X_std)
X_train_pca = pca.transform(X_std)
X_test_pca = pca.transform(X_std_test)
clf = SVC(C=60,kernel=rbf,random_state=42)

scores=cross_val_score(clf, X_train_pca,Y, cv=10)
print(scores)
clf.fit(X_train_pca,Y)
clf_predictions = clf.predict(X_test_pca)
print(clf_predictions)

time= time.clock() - start_time
print(time)

#### PCA=50, SVM(C=60,kernel=rbf)


In [ ]:
#Implementing PCA with SVM for 60000 samples for 50 features
start_time = time.clock()

pca = PCA(n_components=50, svd_solver='randomized',whiten=True,random_state=42)
pca.fit(X_std)
X_train_pca = pca.transform(X_std)
X_test_pca = pca.transform(X_std_test)

clf = SVC(C = 60, kernel = 'rbf')
scores=cross_val_score(clf, X_train_pca,Y, cv=10)
print("Mean accuracy score when C = {0} is {1}".format(60,scores.mean()))
clf.fit(X_train_pca, Y)

predictions = clf.predict(X_test_pca)

#### PCA=50, SVM(C = 20, kernel = 'rbf')

In [ ]:
#Implementing PCA with SVM for 60000 samples for 50 features 
start_time = time.clock()
pca = PCA(n_components=50,random_state=42)

pca.fit(X_std)
X_train_pca = pca.transform(X_std)
X_test_pca = pca.transform(X_std_test)
clf = SVC(C = 20, kernel = 'rbf',random_state=42)
scores=cross_val_score(clf, X_train_pca,Y, cv=10)
print("Mean accuracy score when C = {0} is {1}".format(20,scores.mean()))

clf.fit(X_train_pca, Y)


predictions = clf.predict(X_test_pca)

print(predictions)

end_time = time.clock() - start_time
print("time in seconds: ",end_time)

### 2. PCA +Random Forest

In [ ]:
X= train_data.iloc[:, 2:786].values
Y= train_data.iloc[:,1].values
print(X.shape)

X_test= test_data.iloc[:,1:].values
print(X_test.shape)

X_std = StandardScaler().fit_transform(X)
X_std_test = StandardScaler().fit_transform(X_test)
print(X_std.shape)

#### PCA=200, Random forest(n_estimators= 800, max_depth=70,max_features='sqrt',random_state=42)

In [ ]:
start_time = time.clock()
pca = PCA(n_components=200,random_state=42)

pca.fit(X_std)
X_train_pca = pca.transform(X_std)
X_test_pca = pca.transform(X_std_test)


rf = RandomForestClassifier(n_estimators= 800, max_depth=70,max_features='sqrt',random_state=42)
rf.fit(X_train_pca, Y)
predictions = rf.predict(X_test_pca)
print(predictions)

time= time.clock() - start_time
print(" Time in mins: ",time)

#### PCA=200, Random forest(n_estimators= 100, max_depth=10)

In [ ]:
start_time = time.clock()
pca = PCA(n_components=50,random_state=42)

pca.fit(X_std)
X_train_pca = pca.transform(X_std)
X_test_pca = pca.transform(X_std_test)


rf = RandomForestClassifier(n_estimators= 100, max_depth=10, random_state=42)
rf.fit(X_train_pca, Y)
predictions = rf.predict(X_test_pca)
print(predictions)

time= time.clock() - start_time
print(" Time in mins: ",time)

### 3. PCA +Gradient Boosting Classifier

#### PCA=200, Default GBC()

In [ ]:
start_time = time.clock()
pca = PCA(n_components=200, svd_solver='randomized',
whiten=True,random_state=42)
pca.fit(X_std)
X_train_pca = pca.transform(X_std)
X_test_pca = pca.transform(X_std_test)
gbgs = GradientBoostingClassifier()
scores=cross_val_score(gbgs, X_train_pca,Y, cv=10)
print(scores)
gbgs.fit(X_train_pca,Y)
#print(gbgs.best_params_)
gbgs_predictions = gbgs.predict(X_test_pca)
print(gbgs_predictions)
time= time.clock() - start_time
print(time)

#### PCA=100, GBC(max_depth=10,n_estimators=100)

In [ ]:

start_time = time.clock()
pca = PCA(n_components=100,random_state=42)

X_train_pca = pca.fit_transform(X_std)
X_test_pca = pca.fit_transform(X_std_test)

gbgs  = GradientBoostingClassifier(max_depth=10,n_estimators=100,random_state=42)


gbgs.fit(X_train_pca,Y)

gbgs_predictions = gbgs.predict(X_test_pca)
print(gbgs_predictions)
time= time.clock() - start_time
print(" Time in mins: ",time)

### LDA Feature Extraction

In [ ]:
#Explained Variance

lda = LinearDiscriminantAnalysis(n_components=4)
X_lda = lda.fit_transform(X,Y)
lda.n_components=4
explained_variance = lda.explained_variance_ratio_
cum_var_explained=np.cumsum(explained_variance)
print(cum_var_explained)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(cum_var_explained)
plt.axis('tight')  
plt.grid() 
plt.xlabel('n_components') 
plt.ylabel('Cumulative_Variance_explained')  
plt.show()

### 1. LDA +SVM

In [ ]:
#Implementing LDA with SVM for 60000 samples for 4 features 

start_time = time.clock()
lda = LinearDiscriminantAnalysis(n_components=4)

lda.fit(X_std,Y)
X_train_lda = lda.transform(X_std)
X_test_lda = lda.transform(X_test)



grid = SVC(C=60,kernel='rbf')

grid.fit(X_train_lda,Y)

scores=cross_val_score(grid, X_train_lda,Y, cv=10)
print(scores)
print(grid_predictions)
time= time.clock() - start_time
print(time)

### 2. LDA+ Random Forest

In [ ]:
start_time = time.clock()
lda = LinearDiscriminantAnalysis(n_components=4)

lda.fit(X_std,Y)
X_train_lda = lda.transform(X_std)
X_test_lda = lda.transform(X_test)
rf = RandomForestClassifier(n_estimators=100,max_depth=10,random_state=42)
scores=cross_val_score(rf, X_train_lda,Y, cv=10)

print("Mean accuracy score when n_estimators = {0} is {1}".format(100,scores.mean()))
rf.fit(X_train_lda,Y )

rf_predictions = rf.predict(X_test_lda)
print(rf_predictions)
time= time.clock() - start_time
print(time)

## ROC for PCA+SVM

### Spliting Training Data into Train and Test

#### a. For SVC(C=60) and PCA=100

In [ ]:
Y= label_binarize(Y, classes=[0,1,2,3,4])
n_classes = 5
X_std = StandardScaler().fit_transform(X)
# shuffle and split training and test sets
X_train, X_train_test, Y_train, Y_train_test =\
    train_test_split(X_std, Y, test_size=0.2, random_state=42)

pca = PCA(n_components=100,random_state=42)
pca.fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

#classifier
clf = OneVsRestClassifier(SVC(C=60,kernel='rbf',random_state=42))
y_score = clf.fit(X_train, Y_train).decision_function(X_train_test)

In [ ]:
# Compute ROC curve and ROC area for each class
import matplotlib.pyplot as plt
fpr = dict()
tpr = dict()
roc_auc = dict()
n_classes=5
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(Y_train_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

colors = ['aqua', 'darkorange', 'cornflowerblue','green','purple']
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))
# Plot of a ROC curve for a specific class
for i, color in zip(range(n_classes), colors):
    plt.figure()

    plt.plot(fpr[i], tpr[i], label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

#### b. For SVC(C=20, Kernel=rbf) and PCA=50

In [ ]:
pca = PCA(n_components=50,random_state=42)
pca.fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

Y= label_binarize(Y, classes=[0,1,2,3,4])
n_classes = 5
X_std = StandardScaler().fit_transform(X)
# shuffle and split training and test sets
X_train, X_train_test, Y_train, Y_train_test =\
    train_test_split(X_std, Y, test_size=0.2, random_state=42)

#classifier
clf = OneVsRestClassifier(SVC(C=20,kernel='rbf',random_state=42))
y_score = clf.fit(X_train, Y_train).decision_function(X_train_test)

In [ ]:
# Compute ROC curve and ROC area for each class
import matplotlib.pyplot as plt
fpr = dict()
tpr = dict()
roc_auc = dict()
n_classes=5
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(Y_train_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

colors = ['aqua', 'darkorange', 'cornflowerblue','green','purple']
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))
# Plot of a ROC curve for a specific class
for i, color in zip(range(n_classes), colors):
    plt.figure()

    plt.plot(fpr[i], tpr[i], label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

## Other metrics on Training Data for PCA 

### PCA with SVM and Random forest

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_std, Y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

####  a. F1 Score and Log Loss for PCA =100 with SVM

In [ ]:
pca = PCA(n_components=100,random_state=42)
pca.fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

clf = SVC(C = 60, kernel = 'rbf', random_state=42)
scores=cross_val_score(clf, X_train_pca,y_train, cv=10)
print("Mean accuracy score when C = {0} is {1}".format(60,scores.mean()))
clf.fit(X_train_pca, y_train)
predictions = clf.predict(X_test_pca)
y_score = clf.fit(X_train_pca, y_train).decision_function(X_test_pca)

f1_s = f1_score(y_test, predictions,average=None)
print("F1 score when C = {0} is {1}".format(60,f1_s))

cf = SVC(C=60, kernel='rbf',probability=True,random_state=42)
cf.fit(X_train_pca, y_train)
svmpred = cf.predict_proba(X_test_pca)
lg_loss = log_loss(y_test, svmpred)
print("Logarithm loss is {0}".format(lg_loss))

####  b. F1 Score and Log Loss for PCA=50+SVM

In [ ]:
pca = PCA(n_components=50,random_state=42)

pca.fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

clf = SVC(C = 20, kernel = 'rbf', random_state=42)
scores=cross_val_score(clf, X_train_pca,y_train, cv=10)
print("Mean accuracy score when C = {0} is {1}".format(20,scores.mean()))

clf.fit(X_train_pca, y_train)
predictions = clf.predict(X_test_pca)

y_score = clf.fit(X_train_pca, y_train).decision_function(X_test_pca)

f1_s = f1_score(y_test, predictions,average=None)
print("F1 score when C = {0} is {1}".format(20,f1_s))

cf = SVC(C=20, kernel='rbf',probability=True,random_state=42)
cf.fit(X_train_pca, y_train)
svmpred = cf.predict_proba(X_test_pca)
lg_loss = log_loss(y_test, svmpred)
print("Logarithm loss is {0}".format(lg_loss))

#### F1 score and Log Loss for PCA+Random Forest

In [ ]:

rf = RandomForestClassifier(n_estimators=100,max_depth=10,random_state=42)
scores=cross_val_score(rf, X_train_pca,y_train, cv=10)
print("Mean accuracy score when n_estimators = {0} is {1}".format(100,scores.mean()))
rf.fit(X_train_pca,y_train )

rf_predictions = rf.predict(X_test_pca)
f1_s = f1_score(y_test, rf_predictions,average=None)
print("F1 score when n_estimators = {0} is {1}".format(100,f1_s))

rfpred = rf.predict_proba(X_test_pca)
lg_loss = log_loss(y_test, rfpred)
print("Logarithm loss is {0}".format(lg_loss))

### LDA with SVM for other metrics

In [ ]:

lda = LinearDiscriminantAnalysis(n_components=4)

lda.fit(X_std,y_train)
X_train_lda = lda.transform(X_train)
X_test_lda = lda.transform(X_test)


#### F1 score and Log loss for LDA+SVM

In [ ]:

clf = SVC(C = 60, kernel = 'rbf', random_state=42)
scores=cross_val_score(clf, X_train_lda,y_train, cv=10)
print("Mean accuracy score when C = {0} is {1}".format(60,scores.mean()))
clf.fit(X_train_lda, y_train)
predictions = clf.predict(X_test_lda)
y_score = clf.fit(X_train_lda, y_train).decision_function(X_test_lda)
f1_s = f1_score(y_test, predictions,average=None)
print("F1 score when C = {0} is {1}".format(60,f1_s))

cf = SVC(C=60, kernel='rbf',probability=True,random_state=42)
cf.fit(X_train_lda, y_train)
svmpred = cf.predict_proba(X_test_lda)
lg_loss = log_loss(y_test, svmpred)
print("Logarithm loss is {0}".format(lg_loss))

# Question 2

## Classification : Convolutional Neural Networks

## Implementation of your Design Choices

### Importing Libraries

In [ ]:
import pandas as pd
import time
import io
import matplotlib.pyplot as plt # Import matplotlib for data visualisation
import seaborn as sns
import random
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.callbacks import TensorBoard


### Extraction DataSet from Kaggle for Part-2 

In [ ]:
from google.colab import files
import io
uploaded = files.upload()
!pip install -q kaggle

# Let's make sure the kaggle.json file is present.
!ls -lha kaggle.json

# Next, install the Kaggle API client.
!pip install -q kaggle

# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c ece657a-w20-asg3-part2

!unzip train.csv.zip
!unzip testX.csv.zip

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('testX.csv')

In [ ]:
# Prepare the training and testing dataset 
X_train= train_data.iloc[:, 2:786].values
y_train= train_data.iloc[:,1].values
print(X_train.shape)
y_train.shape

X_test= test_data.iloc[:,1:].values
#Y_test =test_data.iloc[:,1].values
X_test.shape

from sklearn.preprocessing import StandardScaler
X_train = StandardScaler().fit_transform(X_train)
X_test = StandardScaler().fit_transform(X_test)

X_train, X_validate, y_train, y_validate = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)

In [ ]:
# * unpack the tuple
X_train = X_train.reshape(X_train.shape[0], *(28, 28, 1))
X_test = X_test.reshape(X_test.shape[0], *(28, 28, 1))
X_validate = X_validate.reshape(X_validate.shape[0], *(28, 28, 1))

## CNN with 1 convolutional layer

In [ ]:
start_time = time.clock()
cnn_model = Sequential()

cnn_model.add(Conv2D(32, (3, 3), input_shape = (28,28,1), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size = (2, 2)))
cnn_model.add(Dropout(0.25))

cnn_model.add(Flatten())
cnn_model.add(Dense(units = 512, activation = 'relu'))
cnn_model.add(Dropout(0.4))
cnn_model.add(Dense(units = 5, activation = 'softmax'))

In [ ]:
cnn_model.compile(loss ='sparse_categorical_crossentropy', optimizer=Adam(),metrics =['accuracy'])

In [ ]:
cnn_model.summary()

In [ ]:
epochs = 50

history = cnn_model.fit(X_train, y_train, batch_size = 128, epochs = epochs,
                        verbose = 1, validation_data = (X_validate, y_validate))

In [ ]:
predicted_classes = cnn_model.predict_classes(X_test)
print(predicted_classes)
time= time.clock() - start_time
print(time)

## Plots for Training Epoch vs  Accuracy and Training Epoch vs Loss

In [ ]:
def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.xlabel('Training Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Training Epoch')
    plt.ylabel('Loss')
    plt.legend()
plot_history(history)

## CNN with 2 Convolutional Layers

In [ ]:
start_time = time.clock()
cnn_model = Sequential()


cnn_model.add(Conv2D(32, (3, 3), input_shape = (28,28,1), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size = (2, 2)))


cnn_model.add(Conv2D(64, (3, 3), input_shape = (28,28,1), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size = (2, 2)))


cnn_model.add(Dropout(0.25))

cnn_model.add(Flatten())
cnn_model.add(Dense(units = 512, activation = 'relu'))
cnn_model.add(Dropout(0.4))
cnn_model.add(Dense(units = 5, activation = 'softmax'))

In [ ]:
cnn_model.compile(loss ='sparse_categorical_crossentropy', optimizer=Adam(),metrics =['accuracy'])

In [ ]:
cnn_model.summary()

In [ ]:
epochs = 50

history = cnn_model.fit(X_train, y_train, batch_size = 128, epochs = epochs,
                        verbose = 1, validation_data = (X_validate, y_validate))

In [ ]:
# get the predictions for the test data
predicted_classes = cnn_model.predict_classes(X_test)
print(predicted_classes)
time= time.clock() - start_time
print(time)

## CNN with 3 convolutional layers

In [ ]:
start_time = time.clock()
cnn_model = Sequential()

# Try 32 fliters first then 64
cnn_model.add(Conv2D(32, (3, 3), input_shape = (28,28,1), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size = (2, 2)))


cnn_model.add(Conv2D(64, (3, 3), input_shape = (28,28,1), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size = (2, 2)))


cnn_model.add(Conv2D(128, (3, 3), input_shape = (28,28,1), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size = (2, 2)))
cnn_model.add(Dropout(0.25))


cnn_model.add(Flatten())
cnn_model.add(Dense(units = 512, activation = 'relu'))
cnn_model.add(Dropout(0.4))
cnn_model.add(Dense(units = 5, activation = 'softmax'))

In [ ]:
cnn_model.compile(loss ='sparse_categorical_crossentropy', optimizer=Adam(),metrics =['accuracy'])

In [ ]:
cnn_model.summary()

In [ ]:
epochs = 50

history = cnn_model.fit(X_train, y_train, batch_size = 128, epochs = epochs,
                        verbose = 1, validation_data = (X_validate, y_validate))

In [ ]:
# get the predictions for the test data
predicted_classes = cnn_model.predict_classes(X_test)
print(predicted_classes)
time= time.clock() - start_time
print(time)

## ResNet- for Epoch=25

In [ ]:
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import MaxPooling2D, AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
#from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('testX.csv')

# Prepare the training and testing dataset 
X_train= train_data.iloc[:, 2:786].values
Y_train= train_data.iloc[:,1].values
X_train.shape

X_test= test_data.iloc[:,1:].values
#Y_test =test_data.iloc[:,1].values
X_test.shape

In [ ]:
#Defining constants
epochs = 25
batch_size = 100
data_augmentation = False
img_size = 28

num_classes = 5
num_filters = 64
num_blocks = 4
num_sub_blocks = 2
use_max_pool = False

In [ ]:
x_train = X_train.reshape(X_train.shape[0],img_size,img_size,1)
x_test = X_test.reshape(X_test.shape[0],img_size,img_size,1)
input_size = (img_size, img_size,1)
# Normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('Y_train shape:', Y_train.shape)

#Converting labels to one-hot vectors
y_train = keras.utils.to_categorical(Y_train, num_classes)

In [ ]:
start_time = time.clock()
#Creating model based on ResNet published archietecture
inputs = Input(shape=input_size)
x = Conv2D(num_filters, padding='same', 
           kernel_initializer='he_normal', 
           kernel_size=7, strides=2,
           kernel_regularizer=l2(1e-4))(inputs)
x = BatchNormalization()(x)
x = Activation('relu')(x)

#Check by applying max pooling later (setting it false as size of image is small i.e. 28x28)
if use_max_pool:
    x = MaxPooling2D(pool_size=3,padding='same', strides=2)(x)
    num_blocks =3
#Creating Conv base stack 

# Instantiate convolutional base (stack of blocks).
for i in range(num_blocks):
    for j in range(num_sub_blocks):
        strides = 1
        is_first_layer_but_not_first_block = j == 0 and i > 0
        if is_first_layer_but_not_first_block:
            strides = 2
        #Creating residual mapping using y
        y = Conv2D(num_filters,
                   kernel_size=3,
                   padding='same',
                   strides=strides,
                   kernel_initializer='he_normal',
                   kernel_regularizer=l2(1e-4))(x)
        y = BatchNormalization()(y)
        y = Activation('relu')(y)
        y = Conv2D(num_filters,
                   kernel_size=3,
                   padding='same',
                   kernel_initializer='he_normal',
                   kernel_regularizer=l2(1e-4))(y)
        y = BatchNormalization()(y)
        if is_first_layer_but_not_first_block:
            x = Conv2D(num_filters,
                       kernel_size=1,
                       padding='same',
                       strides=2,
                       kernel_initializer='he_normal',
                       kernel_regularizer=l2(1e-4))(x)
        #Adding back residual mapping
        x = keras.layers.add([x, y])
        x = Activation('relu')(x)

    num_filters = 2 * num_filters

# Add classifier on top.
x = AveragePooling2D()(x)
y = Flatten()(x)
outputs = Dense(num_classes,
                activation='softmax',
                kernel_initializer='he_normal')(y)

# Instantiate and compile model.
model = Model(inputs=inputs, outputs=outputs)
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])
model.summary()

In [ ]:
 history = model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_split = 0.2)

In [ ]:
pred=model.predict(x_test)
pred=np.argmax(pred,axis=1)
time= time.clock() - start_time
print(time)
df=pd.DataFrame(pred)
df.index+=0
df.index.name='Id'
df.columns=['Label']
df.to_csv('resnet1.csv',header='true')

## Plots for Training Epoch vs  Accuracy and Training Epoch vs Loss

In [ ]:
def plot_history(history):
    acc = history.history['accuracy']
    #val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    #val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    #plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training accuracy')
    plt.xlabel('Training Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    #plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training loss')
    plt.xlabel('Training Epoch')
    plt.ylabel('Loss')
    plt.legend()
plot_history(history)

## ResNet- Epoch=5

In [ ]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('testX.csv')

# Prepare the training and testing dataset 
X_train= train_data.iloc[:, 2:786].values
Y_train= train_data.iloc[:,1].values
X_train.shape

X_test= test_data.iloc[:,1:].values
#Y_test =test_data.iloc[:,1].values
X_test.shape

In [ ]:
#Defining constants
epochs = 25
batch_size = 100
data_augmentation = False
img_size = 28

num_classes = 5
num_filters = 64
num_blocks = 4
num_sub_blocks = 2
use_max_pool = False

In [ ]:
x_train = X_train.reshape(X_train.shape[0],img_size,img_size,1)
x_test = X_test.reshape(X_test.shape[0],img_size,img_size,1)
input_size = (img_size, img_size,1)
# Normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('Y_train shape:', Y_train.shape)

#Converting labels to one-hot vectors
y_train = keras.utils.to_categorical(Y_train, num_classes)

In [ ]:
start_time = time.clock()
#Creating model based on ResNet published archietecture
inputs = Input(shape=input_size)
x = Conv2D(num_filters, padding='same', 
           kernel_initializer='he_normal', 
           kernel_size=7, strides=2,
           kernel_regularizer=l2(1e-4))(inputs)
x = BatchNormalization()(x)
x = Activation('relu')(x)

#Check by applying max pooling later (setting it false as size of image is small i.e. 28x28)
if use_max_pool:
    x = MaxPooling2D(pool_size=3,padding='same', strides=2)(x)
    num_blocks =3
#Creating Conv base stack 

# Instantiate convolutional base (stack of blocks).
for i in range(num_blocks):
    for j in range(num_sub_blocks):
        strides = 1
        is_first_layer_but_not_first_block = j == 0 and i > 0
        if is_first_layer_but_not_first_block:
            strides = 2
        #Creating residual mapping using y
        y = Conv2D(num_filters,
                   kernel_size=3,
                   padding='same',
                   strides=strides,
                   kernel_initializer='he_normal',
                   kernel_regularizer=l2(1e-4))(x)
        y = BatchNormalization()(y)
        y = Activation('relu')(y)
        y = Conv2D(num_filters,
                   kernel_size=3,
                   padding='same',
                   kernel_initializer='he_normal',
                   kernel_regularizer=l2(1e-4))(y)
        y = BatchNormalization()(y)
        if is_first_layer_but_not_first_block:
            x = Conv2D(num_filters,
                       kernel_size=1,
                       padding='same',
                       strides=2,
                       kernel_initializer='he_normal',
                       kernel_regularizer=l2(1e-4))(x)
        #Adding back residual mapping
        x = keras.layers.add([x, y])
        x = Activation('relu')(x)

    num_filters = 2 * num_filters

# Add classifier on top.
x = AveragePooling2D()(x)
y = Flatten()(x)
outputs = Dense(num_classes,
                activation='softmax',
                kernel_initializer='he_normal')(y)

# Instantiate and compile model.
model = Model(inputs=inputs, outputs=outputs)
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])
model.summary()

In [ ]:
 history = model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_split = 0.2)

In [ ]:
pred=model.predict(x_test)
pred=np.argmax(pred,axis=1)
time= time.clock() - start_time
print(time)
df=pd.DataFrame(pred)
df.index+=0
df.index.name='Id'
df.columns=['Label']
df.to_csv('resnet1.csv',header='true')

## CNN Other Metrics with 1 convolutional layer - Classification Report

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import cross_val_score
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
from sklearn.metrics import classification_report

In [ ]:
train_data = pd.read_csv('train.csv')

# Prepare the training and testing dataset 
X= train_data.iloc[:, 2:786].values
y= train_data.iloc[:,1].values
print(X.shape)
y.shape

In [ ]:

X_std = StandardScaler().fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(X_std, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
X_train = X_train.reshape(X_train.shape[0], *(28, 28, 1))
X_test = X_test.reshape(X_test.shape[0], *(28, 28, 1))

In [ ]:
cnn_model = Sequential()

# Try 32 fliters first then 64
cnn_model.add(Conv2D(32, (3, 3), input_shape = (28,28,1), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size = (2, 2)))

cnn_model.add(Dropout(0.25))


cnn_model.add(Flatten())
cnn_model.add(Dense(units = 512, activation = 'relu'))
cnn_model.add(Dropout(0.4))
cnn_model.add(Dense(units = 5, activation = 'softmax'))

cnn_model.compile(loss ='sparse_categorical_crossentropy', optimizer=Adam(),metrics =['accuracy'])
cnn_model.summary()

In [ ]:
epochs = 50

history = cnn_model.fit(X_train, y_train, batch_size = 128, epochs = epochs,
                        verbose = 1, validation_split = 0.2)

In [ ]:
score = cnn_model.evaluate(X_test, y_test)

print('Loss: {:.4f}'.format(score[0]))
print('Accuracy: {:.4f}'.format(score[1]))

In [ ]:
classes = ['0','1','2','3','4']
print(classification_report(y_test, predicted_classes, target_names = classes))

## ResNet Other Metrics for Epoch=25 - Classification Report

In [ ]:
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import MaxPooling2D, AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
#from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from sklearn.metrics import classification_report
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('testX.csv')

# Prepare the training and testing dataset 
X= train_data.iloc[:, 2:786].values
Y= train_data.iloc[:,1].values
print(X.shape)
Y.shape

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

#Defining constants
epochs = 25
batch_size = 100
data_augmentation = False
img_size = 28

num_classes = 5
num_filters = 64
num_blocks = 4
num_sub_blocks = 2
use_max_pool = False

In [ ]:
x_train = X_train.reshape(X_train.shape[0],img_size,img_size,1)
x_test = X_test.reshape(X_test.shape[0],img_size,img_size,1)
input_size = (img_size, img_size,1)
# Normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('Y_train shape:', y_train.shape)

#Converting labels to one-hot vectors
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
#Creating model based on ResNet published archietecture
inputs = Input(shape=input_size)
x = Conv2D(num_filters, padding='same', 
           kernel_initializer='he_normal', 
           kernel_size=7, strides=2,
           kernel_regularizer=l2(1e-4))(inputs)
x = BatchNormalization()(x)
x = Activation('relu')(x)

#Check by applying max pooling later (setting it false as size of image is small i.e. 28x28)
if use_max_pool:
    x = MaxPooling2D(pool_size=3,padding='same', strides=2)(x)
    num_blocks =3
#Creating Conv base stack 

# Instantiate convolutional base (stack of blocks).
for i in range(num_blocks):
    for j in range(num_sub_blocks):
        strides = 1
        is_first_layer_but_not_first_block = j == 0 and i > 0
        if is_first_layer_but_not_first_block:
            strides = 2
        #Creating residual mapping using y
        y = Conv2D(num_filters,
                   kernel_size=3,
                   padding='same',
                   strides=strides,
                   kernel_initializer='he_normal',
                   kernel_regularizer=l2(1e-4))(x)
        y = BatchNormalization()(y)
        y = Activation('relu')(y)
        y = Conv2D(num_filters,
                   kernel_size=3,
                   padding='same',
                   kernel_initializer='he_normal',
                   kernel_regularizer=l2(1e-4))(y)
        y = BatchNormalization()(y)
        if is_first_layer_but_not_first_block:
            x = Conv2D(num_filters,
                       kernel_size=1,
                       padding='same',
                       strides=2,
                       kernel_initializer='he_normal',
                       kernel_regularizer=l2(1e-4))(x)
        #Adding back residual mapping
        x = keras.layers.add([x, y])
        x = Activation('relu')(x)

    num_filters = 2 * num_filters

# Add classifier on top.
x = AveragePooling2D()(x)
y = Flatten()(x)
outputs = Dense(num_classes,
                activation='softmax',
                kernel_initializer='he_normal')(y)

# Instantiate and compile model.
model = Model(inputs=inputs, outputs=outputs)
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_split = 0.2)

In [ ]:
pred=model.predict(x_test)
pred=np.argmax(pred,axis=1)
print(pred)

In [ ]:
score = model.evaluate(x_test, y_test, verbose=1)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
classes = ['0','1','2','3','4']
test_label=np.argmax(y_test, axis=1)
test_label[1]
print(classification_report(test_label, pred, target_names=classes))

## Reference

<ol>
    Question 1:
<li>https://statinfer.com/204-6-8-svm-advantages-disadvantages-applications/</li>
<li>https://www.knowledgehut.com/blog/data-science/linear-discriminant-analysis-for-machine-learning</li>
<li>https://stackoverflow.com/questions/45332410/sklearn-roc-for-multiclass-classification</li>
<li> https://www.i2tutorials.com/top-machine-learning-interview-questions-and-answers/what-are-the-pros-and-cons-of-the-pca/</li>
<li>https://medium.com/machine-learning-researcher/dimensionality-reduction-pca-and-lda-6be91734f567</li>
<li>https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1\_score.html</li>
<li> https://medium.com/analytics-vidhya/accuracy-vs-f1-score-6258237beca2</li>
<li>https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6
</li>
<li> https://scikit-learn.org/stable/modules/generated/sklearn.metrics.log\_loss.html</li>
<li> https://stackoverflow.com/questions/33616102/sklearn-log-loss-different-number-of-classes/42953278</li>
<li> https://towardsdatascience.com/https-medium-com-pupalerushikesh-svm-f4b42800e989</li>
<li>https://chrisalbon.com/machine\_learning/support\_vector\_machines/svc\_parameters\_using\_rbf\_kernel/</li>
<li>https://www.kdnuggets.com/2016/06/select-support-vector-machine-kernels.html</li>
<li>https://towardsdatascience.com/visualising-high-dimensional-datasets-using-pca-and-t-sne-in-python-8ef87e7915b</li>
<li> https://data-flair.training/blogs/gradient-boosting-algorithm/</li>
<li>https://scikit-learn.org/stable/modules/ensemble.html#gradient-boosting</li>
<li>https://stats.stackexchange.com/questions/258938/pca-before-random-forest-regression-provide-better-predictive-scores-for-my-data/258942</li>
<li> https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74</li>
<li> https://towardsdatascience.com/machine-learning-step-by-step-6fbde95c455a</li>
<li>https://www.kaggle.com/residentmario/dimensionality-reduction-and-pca-for-fashion-mnist</li>
<li>https://www.kaggle.com/lonewolf95/classification-tutorial-with-pca-and-gridsearchcv</li>
<li>https://towardsdatascience.com/an-implementation-and-explanation-of-the-random-forest-in-python-77bf308a9b76</li>
<li>https://github.com/dask/dask-ml/issues/159</li>
<li> https://towardsdatascience.com/hyperparameter-optimization-with-keras-b82e6364ca53</li>
<li>https://machinelearningmastery.com/make-predictions-scikit-learn/</li>
<li>https://stackabuse.com/implementing-pca-in-python-with-scikit-learn/</li>
<li>https://medium.com/@sarayupgouda/pca-principal-component-analysis-in-python-f9836c25acb9 https://stackabuse.com/implementing-pca-in-python-with-scikit-learn/</li>
<li> https://go2analytics.wordpress.com/2016/07/26/implement-classification-in-python-and-roc-plotting-svc-example/</li>
    
    
</ol>    
    Question 2:
<li>https://www.machinecurve.com/index.php/2019/09/17/how-to-create-a-cnn-classifier-with-keras/</li>
<li>https://pravarmahajan.github.io/fashion/</li>
<li>https://github.com/cmasch/zalando-fashion-mnist</li>
<li>https://github.com/zalandoresearch/fashion-mnist</li>
<li>http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/</li>
<li>https://github.com/nuclearczy/SVM\_and\_CNN\_on\_Fashion\_MNIST\_Dataset</li>
<li>https://github.com/shoji9x9/Fashion-MNIST-By-ResNet/blob/master/Fashion-MNIST-by-ResNet-50.ipynb</li>
<li>https://towardsdatascience.com/understanding-and-coding-a-resnet-in-keras-446d7ff84d33</li>
<li>https://www.kaggle.com/girishgupta/fashion-mnist-using-resnet</li>
<li>https://towardsdatascience.com/introduction-to-resnets-c0a830a288a4</li>
<li>https://stackoverflow.com/questions/54589669/confusion-matrix-error-classification-metrics-cant-handle-a-mix-of-multilabel</li>
<li>https://www.kaggle.com/akumaldo/resnet-from-scratch-keras</li>
<li>https://towardsdatascience.com/metrics-to-evaluate-your-machine-learning-algorithm-f10ba6e38234</li>
<li>https://towardsdatascience.com/the-4-convolutional-neural-network-models-that-can-classify-your-fashion-images-9fe7f3e5399d</li>
<li>https://keras.io/</li>
<li>https://bmcproc.biomedcentral.com/articles/10.1186/1753-6561-5-S3-S11</li>
<li>https://towardsdatascience.com/understanding-and-coding-a-resnet-in-keras-446d7ff84d33</li>
<li>https://machinelearningmastery.com/understand-the-dynamics-of-learning-rate-on-deep-learning-neural-networks/</li>
<li>https://towardsdatascience.com/epoch-vs-iterations-vs-batch-size-4dfb9c7ce9c9</li>
<li>https://www.kaggle.com/faressayah/fashion-classification-mnist-cnn-tutorial</li>


</ol>


